In [1]:
!pip install datasets --quiet
!pip install accelerate -U --quiet
!pip install evaluate --quiet
!pip install jiwer -U  -q
!pip install transformers -U -q
!pip install wandb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 52.4 MB/s eta 0

In [2]:
!git clone https://github.com/f90/jamendolyrics.git --quiet

In [3]:
!gdown 13bUduvbSf4AowpM5NN-owNKUYdQsrfm_
!unzip -q /content/musan_resampled.zip
!rm /content/musan_resampled.zip

Downloading...
From (original): https://drive.google.com/uc?id=13bUduvbSf4AowpM5NN-owNKUYdQsrfm_
From (redirected): https://drive.google.com/uc?id=13bUduvbSf4AowpM5NN-owNKUYdQsrfm_&confirm=t&uuid=0dfffc24-9af3-4568-8d01-00864f522789
To: /content/musan_resampled.zip
100% 1.61G/1.61G [00:13<00:00, 115MB/s]


In [4]:
# !wget https://www.openslr.org/resources/17/musan.tar.gz
# !tar -zxf musan.tar.gz
# !rm -rf /content/musan/music
# !rm -rf /content/musan/speech
# !rm /content/musan.tar.gz

In [1]:
import torch
import argparse
import evaluate
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from datasets import DatasetDict, Audio, load_from_disk, concatenate_datasets
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback
from datasets import load_dataset, DatasetDict
import wandb
import os
import numpy as np
import random
import torchaudio
import soundfile
from torch.utils.data import Dataset
import glob
import pandas as pd
import csv
import librosa
import torch.nn as nn
import torch.nn.functional as F
import copy

In [6]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
MODEL_NAME = "openai/whisper-medium"
# LANGUAGE = "en"
FREEZE_FEATURE_ENCODER = False
FREEZE_ENCODER = False

feature_extractor = WhisperFeatureExtractor.from_pretrained(MODEL_NAME)
tokenizer = WhisperTokenizer.from_pretrained(MODEL_NAME,  task="transcribe")
processor = WhisperProcessor.from_pretrained(MODEL_NAME,task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
if FREEZE_FEATURE_ENCODER:
    model.freeze_feature_encoder()

if FREEZE_ENCODER:
    model.freeze_encoder()
    model.model.encoder.gradient_checkpointing = False

In [4]:
metadata = pd.read_csv('/content/jamendolyrics/JamendoLyrics.csv')
metadata.head()

,URL,Filepath,Artist,Title,Genre,LicenseType,Language,LyricOverlap,Polyphonic,NonLexical
0,https://www.jamendo.com/track/1559261/give-me-...,HILA_-_Give_Me_the_Same.mp3,HILA,Give Me The Same,Pop,BY-ND,English,False,False,False
1,https://www.jamendo.com/track/1552064/keep-on,Quentin_Hannappe_-_Keep_On.mp3,Quentin Hannappe,Keep On,Pop,BY-NC-ND,English,False,False,False
2,https://www.jamendo.com/track/1537288/back-in-...,Songwriterz_-_Back_In_Time.mp3,Songwriterz,Back In Time,Pop,BY-ND,English,False,False,True
3,https://www.jamendo.com/track/1442030/peyote,Kinematic_-_Peyote.mp3,KINEMATIC,Peyote,Rock,BY-ND,English,False,True,False
4,https://www.jamendo.com/track/1465148/embers,Avercage_-_Embers.mp3,Avercage,Embers,Rock,BY-NC-SA,English,False,False,False


In [5]:
metadata = metadata.set_index("Filepath")

In [6]:


SR = 16000
mp3_path = '/content/jamendolyrics/mp3/'
csv_path = '/content/jamendolyrics/annotations/lines/'
file_names = sorted(glob.glob(csv_path + '*.csv'))

gather_n = 4 ################################################### Combine n data samples into one data
len_files = []
start_time = []
end_time = []
audio_types = []
lyrics = []
languages = []
audio_dict = {} # load audio array in advance to use less RAM. key: 0~78, value: np.array

for f in file_names:
    c = pd.read_csv(f)
    len_files.append(len(c))

def preprocess_dataset():
    for num, filename in enumerate(file_names):
        len_csv = len_files[num]
        with open(f"{filename}", "r") as f:
            csv_data = csv.reader(f)
            audio_name = filename.split('/')[-1][:-4] + '.mp3'
            language = metadata.loc[audio_name]["Language"].lower()
            processor.tokenizer.set_prefix_tokens(language=language)

            for i, line in enumerate(csv_data):
                if i== 0: # except first head
                    continue

                if gather_n == 1: # not combine samples
                    st = float(line[0])
                    et = float(line[1])
                    lyric = line[2]
                    lyric = processor.tokenizer(lyric).input_ids

                    start_time.append(st)
                    end_time.append(et)
                    lyrics.append(lyric)
                    audio_types.append(num)

                else:
                    if i%gather_n == 1: # start gathering frames
                        st = float(line[0])
                        et = float(line[1])
                        lyric = line[2]

                    else:
                        et = float(line[1])
                        lyric += (' ' + line[2])

                    if i%gather_n == 0 or i == len_csv: # end of the group or the last line
                        start_time.append(st)
                        end_time.append(et)
                        lyric = processor.tokenizer(lyric).input_ids
                        lyrics.append(lyric)
                        audio_types.append(num)

        f.close()

In [7]:
preprocess_dataset()

In [8]:
# Load original audio arrays in advance
for i, filename in enumerate(file_names):
    audio_name = filename.split('/')[-1][:-4] + '.mp3'
    audio_dict[i], _ = librosa.load(mp3_path + audio_name, sr=SR)

trimmed_audios = []
def trim_audio(audio_name, st, et):
    start_frame = SR*st
    end_frame = SR*et
    audio  = audio_dict[audio_name] # sr=44100
    trimmed_audio = audio[int(start_frame):int(end_frame)+1]

    return trimmed_audio

In [9]:
for audio_type, st, et in zip(audio_types, start_time, end_time):
    trimmed_audios.append(trim_audio(audio_type, st, et))

In [10]:
# noise_ds = load_dataset('./musan')
noise_ds = load_from_disk('/content/content/musan_resampled')

In [11]:
trimmed_audios[0]

array([-0.03685588, -0.12578191,  0.02261727, ...,  0.18195425,
        0.09405375,  0.03368825], dtype=float32)

In [12]:
# Pytorch Dataset
# Have to add noise before feature extraction


def add_noise_to_audio(audio, noise, clean_db, noise_snr):
    noise_db = 10 * np.log10(np.mean(noise ** 2)+1e-4)
    noises = [np.sqrt(10 ** ((clean_db - noise_db - noise_snr) / 10)) * noise]

    if len(noises[0]) < len(audio):
        added = np.pad(noises[0], (0, len(audio)-len(noises[0])), 'constant') + audio
    else:
        added = audio + noises[0][:len(audio)]

    return added

class PairedDataset(Dataset):
    def __init__(self, trimmed_audios, lyrics,noise_ds,snr = 10,random_augment = True):
        self.lyrics = lyrics
        self.trimmed_audios = trimmed_audios
        self.noise_dataset = noise_ds
        self.feature_extractor = feature_extractor
        self.snr = snr

        self.random_augment = random_augment

    def __len__(self):
        return len(self.lyrics)

    def __getitem__(self, index):
        if self.random_augment:
          noise_index = random.randint(0, len(self.noise_dataset) - 1)
        else:
          noise_index = index % len(self.noise_dataset)

        noise_sample = self.noise_dataset[noise_index]
        noise_array = noise_sample['audio']['array'] * 30
        audio_array = self.trimmed_audios[index]

        clean_db = 10 * np.log10(np.mean(audio_array ** 2)+1e-4) ##
        audio_array = audio_array.squeeze()#.numpy()
        noisy_audio = add_noise_to_audio(audio_array, noise_array, clean_db, noise_snr=self.snr)

        audio_processed = self.feature_extractor(noisy_audio, sampling_rate=SR).input_features[0]

        return {'input_features': audio_processed, 'labels': self.lyrics[index]}




In [13]:
SNR = -3
paired_dataset = PairedDataset(trimmed_audios, lyrics, noise_ds,snr = SNR)

### Approach 1. Use pytorch dataset format (I'm not sure if this format is also available in huggingface Trainer)

In [14]:
### Approach 1. Use pytorch dataset format (I'm not sure if this format is also available in huggingface Trainer)

import torch
from torch.utils.data.dataset import random_split
len_full_dataset = len(paired_dataset)

# train samples
train_p = 0.8
len_train = int(len_full_dataset * train_p)

# valid samples
valid_p = 0.1
len_valid = int(len_full_dataset * 0.1)

train_dataset, valid_dataset = random_split(paired_dataset, [len_train, len_full_dataset-len_train])
valid_dataset, test_dataset = random_split(valid_dataset, [len_valid, len(valid_dataset)-len_valid])

train_dataset.random_augment = True
valid_dataset.random_augment = False
test_dataset.random_augment = False

In [15]:
# # Check if the audio aligns with text well
# import IPython.display as ipd
# idx = 20
# print(train_dataset[idx]['labels'])
# ipd.Audio(train_dataset[idx]['input_features'], rate=44100)

In [16]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt", padding=True)
        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [17]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [18]:
metric = evaluate.load("wer")
DO_NORMALIZE_EVAL = True
normalizer = BasicTextNormalizer()
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    if DO_NORMALIZE_EVAL:
        pred_str = [normalizer(pred) for pred in pred_str]
        label_str = [normalizer(label) for label in label_str]

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

In [19]:
from transformers.integrations import WandbCallback


def decode_predictions(tokenizer, predictions):
    pred_ids = predictions.predictions
    label_ids = predictions.label_ids
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    return {"labels": label_str, "predictions": pred_str}


class WandbPredictionProgressCallback(WandbCallback):
    """Custom WandbCallback to log model predictions during training.

    This callback logs model predictions and labels to a wandb.Table at each logging step during training.
    It allows to visualize the model predictions as the training progresses.

    Attributes:
        trainer (Trainer): The Hugging Face Trainer instance.
        tokenizer (AutoTokenizer): The tokenizer associated with the model.
        sample_dataset (Dataset): A subset of the validation dataset for generating predictions.
        num_samples (int, optional): Number of samples to select from the validation dataset for generating predictions. Defaults to 100.
    """

    def __init__(self, trainer, tokenizer, val_dataset, num_samples=100, freq=2):
        """Initializes the WandbPredictionProgressCallback instance.

        Args:
            trainer (Trainer): The Hugging Face Trainer instance.
            tokenizer (AutoTokenizer): The tokenizer associated with the model.
            val_dataset (Dataset): The validation dataset.
            num_samples (int, optional): Number of samples to select from the validation dataset for generating predictions. Defaults to 100.
            freq (int, optional): Control the frequency of logging. Defaults to 2.
        """
        super().__init__()
        self.trainer = trainer
        self.tokenizer = tokenizer
        noise_indexes = random.sample(range(0, len(val_dataset)-1), num_samples)
        self.sample_dataset = [val_dataset[noise_index] for noise_index in noise_indexes]
        self.freq = freq


    def on_evaluate(self, args, state, control,  **kwargs):
        super().on_evaluate(args, state, control, **kwargs)
        # control the frequency of logging by logging the predictions every `freq` epochs
        if state.epoch % self.freq == 0:
            # generate predictions
            predictions = self.trainer.predict(self.sample_dataset)
            # decode predictions and labels
            predictions = decode_predictions(self.tokenizer, predictions)
            # add predictions to a wandb.Table
            predictions_df = pd.DataFrame(predictions)
            predictions_df["epoch"] = state.epoch

            # get audio samples from sample_dataset

            # add audio samples as a new column to the DataFrame

            records_table = self._wandb.Table(dataframe=predictions_df)
            # log the table to wandb
            self._wandb.log({"sample_predictions": records_table})


In [20]:
os.environ["WANDB_PROJECT"] = "Noisy Lyrics Transcription"  # name your W&B project
os.environ["ENTITY"] = "gct634"  # name your W&B project

os.environ["WANDB_LOG_MODEL"] = "end"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"



In [21]:
#OUTPUT_DIR = "whisper-medium-finetuned-jamendo-snr-3"
OUTPUT_DIR = "whisper-medium-finedtuned-jamendo-snr-3-Kwangbin_testing"
TRAIN_BATCHSIZE = 32
LEARNING_RATE = 1e-5
WARMUP = 5
EVAL_BATCHSIZE = 5
RESUME_FROM_CKPT = None
NUM_EPOCHS = 50
GRADIENT_CHECKPOINTING= True

In [22]:
training_args = Seq2SeqTrainingArguments(
        output_dir= OUTPUT_DIR,
        per_device_train_batch_size=TRAIN_BATCHSIZE,
        gradient_accumulation_steps=1,
        learning_rate=LEARNING_RATE,
        warmup_steps=WARMUP,
        gradient_checkpointing=GRADIENT_CHECKPOINTING,
        fp16=True,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        load_best_model_at_end = True,
        num_train_epochs=NUM_EPOCHS,
        per_device_eval_batch_size=EVAL_BATCHSIZE,
        predict_with_generate=True,
        generation_max_length=225,
        logging_strategy='epoch',
        report_to=["wandb"],
        metric_for_best_model="wer",
        greater_is_better=False,
        optim="adamw_hf",
        resume_from_checkpoint=RESUME_FROM_CKPT,
    )

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics
)


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
torch.cuda.empty_cache()

In [26]:
# We got model and processor
class NoisyEvaluation(WandbCallback):
    def __init__(self, audio_without_noises_path, audio_with_noises_path, model, processor):
        super().__init__()
        self.model = model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        self.processor = processor

        # Load and normalize audio data
        audio_without_noise, sr = librosa.load(audio_without_noises_path, sr=16000)
        audio_without_noise = audio_without_noise / np.max(np.abs(audio_without_noise))
        self.audio_without_noise_input = processor(audio_without_noise, sampling_rate=sr, return_tensors="pt").input_features

        audio_with_noise, sr = librosa.load(audio_with_noises_path, sr=16000)
        audio_with_noise = audio_with_noise / np.max(np.abs(audio_with_noise))
        self.audio_with_noise_input = processor(audio_with_noise, sampling_rate=sr, return_tensors="pt").input_features

        # Move inputs to the same device as the model
        self.audio_without_noise_input = self.audio_without_noise_input.to(self.model.device)
        self.audio_with_noise_input = self.audio_with_noise_input.to(self.model.device)

        # The decoder input typically starts with the BOS token for Whisper
        bos_token_id = processor.tokenizer.bos_token_id
        self.decoder_input_ids = torch.tensor([[bos_token_id]], device=self.model.device)

        # Initial activations storage
        self.activations_without_noise = {}
        self.activations_with_noise = {}
        self.attentions_without_noise = {}
        self.attentions_with_noise = {}

        # Capture initial activations
        self.capture_initial_activations()

    def get_activation(self, name, store_dict):
        def hook(model, input, output):
            store_dict[name] = output.detach()
        return hook

    def get_attention_activation(self, name, store_dict):
        def hook(model, input, output):
            if 'layer_norm' in name.lower() or 'norm' in name.lower():
                store_dict[name] = output.detach()
            else:
                if not isinstance(output, tuple):
                    output = (output,)
                if len(output) > 0 and output[0] is not None:
                    store_dict[name] = output[0].detach()
                else:
                    raise IndexError(f"Expected first element in output tuple for {name}, but got {output}.")
        return hook

    def run_model_and_capture_activations(self, input_features, activations_store, attentions_store):
        hooks = []
        for name, layer in self.model.named_modules():
            if isinstance(layer, nn.Conv1d):
                hooks.append(layer.register_forward_hook(self.get_activation(name, activations_store)))
            elif 'self_attn' in name.lower():  # Assumption: attention layers contain 'attention' in their name
                hooks.append(layer.register_forward_hook(self.get_attention_activation(name, attentions_store)))


        self.model.eval()
        with torch.no_grad():
            output = self.model(input_features=input_features, decoder_input_ids=self.decoder_input_ids)
        for hook in hooks:
            hook.remove()
        return output

    def mean_squared_difference(self, matrix1, matrix2):
        difference = matrix1 - matrix2
        squared_difference = difference ** 2
        mean_squared_diff = torch.mean(squared_difference)
        return mean_squared_diff

    def calculate_entropy(self, tensor):
        # Ensure the tensor is not empty
        if tensor.numel() == 0:
            return float('nan')

        # Normalize tensor to avoid very large/small values
        tensor = tensor / tensor.max(dim=1, keepdim=True)[0]

        # Flatten the tensor to apply log_softmax along the right dimension
        tensor = tensor.flatten(start_dim=1)

        # Apply log_softmax
        log_softmax_tensor = F.log_softmax(tensor, dim=1)

        # Calculate entropy
        exp_log_softmax_tensor = torch.exp(log_softmax_tensor)
        entropy = -torch.sum(exp_log_softmax_tensor * log_softmax_tensor, dim=1)

        # Average the entropy
        average_entropy = torch.mean(entropy)

        # Check for NaNs in the entropy calculation
        if torch.isnan(average_entropy):
            return float('nan')

        return average_entropy.item()
    def capture_initial_activations(self):
        torch.cuda.empty_cache()  # Free up memory before capturing new activations
        original_output_without_noise = self.run_model_and_capture_activations(self.audio_without_noise_input, self.activations_without_noise, self.attentions_without_noise)
        original_output_with_noise = self.run_model_and_capture_activations(self.audio_with_noise_input, self.activations_with_noise, self.attentions_with_noise)

        # Debug: print out captured keys for attention layers
        print("Captured attention keys without noise:", list(self.attentions_without_noise.keys()))
        print("Captured attention keys with noise:", list(self.attentions_with_noise.keys()))



        self.original_conv1_entropy_with_noise = self.calculate_entropy(self.activations_with_noise['model.encoder.conv1'])
        self.original_conv1_entropy_without_noise = self.calculate_entropy(self.activations_without_noise['model.encoder.conv1'])
        self.original_conv2_entropy_with_noise = self.calculate_entropy(self.activations_with_noise['model.encoder.conv2'])
        self.original_conv2_entropy_without_noise = self.calculate_entropy(self.activations_without_noise['model.encoder.conv2'])
        self.original_MSD_conv1 = self.mean_squared_difference(self.activations_with_noise['model.encoder.conv1'], self.activations_without_noise['model.encoder.conv1'])
        self.original_MSD_conv2 = self.mean_squared_difference(self.activations_with_noise['model.encoder.conv2'], self.activations_without_noise['model.encoder.conv2'])
        self.original_MSD_attention0 = self.mean_squared_difference(self.attentions_with_noise['model.encoder.layers.0.self_attn'], self.attentions_without_noise['model.encoder.layers.0.self_attn'])
        self.original_MSD_attention23 = self.mean_squared_difference(self.attentions_with_noise['model.encoder.layers.23.self_attn'], self.attentions_without_noise['model.encoder.layers.23.self_attn'])

        print(f"Baseline Model Convolutional Entropy (With Noise): {self.original_conv1_entropy_with_noise}")
        print(f"Baseline Model Convolutional Entropy (Without Noise): {self.original_conv1_entropy_without_noise}")
        print(f"Baseline Model Convolutional Entropy 2 (With Noise): {self.original_conv2_entropy_with_noise}")
        print(f"Baseline Model Convolutional Entropy 2 (Without Noise): {self.original_conv2_entropy_without_noise}")
        print("Baseline Mean Squared Difference in Conv1: ", self.original_MSD_conv1)
        print("Baseline Mean Squared Difference in Conv2: ", self.original_MSD_conv2)
        print("Baseline Mean Squared Difference in Attention0: ", self.original_MSD_attention0)
        print("Baseline Mean Squared Difference in Attention23: ", self.original_MSD_attention23)

    def on_epoch_end(self, args, state, control, **kwargs):
        super().on_epoch_end(args, state, control, **kwargs)
        activations_without_noise = {}
        activations_with_noise = {}
        attentions_without_noise = {}
        attentions_with_noise = {}

        torch.cuda.empty_cache()  # Free up memory before capturing new activations

        # Run the model without noise and capture activations
        output_without_noise = self.run_model_and_capture_activations(self.audio_without_noise_input, activations_without_noise,  attentions_without_noise)

        # Run the model with noise and capture activations
        output_with_noise = self.run_model_and_capture_activations(self.audio_with_noise_input, activations_with_noise, attentions_with_noise)

        print(f"Baseline Convolutional Entropy (With Noise): {self.original_conv1_entropy_with_noise}")
        print(f"Baseline Convolutional Entropy (Without Noise): {self.original_conv1_entropy_without_noise}")
        print(f"Baseline Convolutional Entropy 2 (With Noise): {self.original_conv2_entropy_with_noise}")
        print(f"Baseline Convolutional Entropy 2 (Without Noise): {self.original_conv2_entropy_without_noise}")
        print("Baseline Mean Squared Difference in Conv1: ", self.original_MSD_conv1)
        print("Baseline Mean Squared Difference in Conv2: ", self.original_MSD_conv2)

        conv1_entropy_with_noise = self.calculate_entropy(activations_with_noise['model.encoder.conv1'])
        print(f"Fine Tuned Convolutional Entropy (With Noise): {conv1_entropy_with_noise}")

        conv1_entropy_without_noise = self.calculate_entropy(activations_without_noise['model.encoder.conv1'])
        print(f"Fine Tuned Convolutional Entropy (Without Noise): {conv1_entropy_without_noise}")

        conv2_entropy_with_noise = self.calculate_entropy(activations_with_noise['model.encoder.conv2'])
        print(f"Fine Tuned Convolutional Entropy 2 (With Noise): {conv2_entropy_with_noise}")

        conv2_entropy_without_noise = self.calculate_entropy(activations_without_noise['model.encoder.conv2'])
        print(f"Fine Tuned Convolutional Entropy 2 (Without Noise): {conv2_entropy_without_noise}")

        MSD_conv1_finetuned = self.mean_squared_difference(activations_with_noise['model.encoder.conv1'], activations_without_noise['model.encoder.conv1'])
        MSD_conv2_finetuned = self.mean_squared_difference(activations_with_noise['model.encoder.conv2'], activations_without_noise['model.encoder.conv2'])

        print("Fine-Tuned Mean Squared Difference in Conv1: ", MSD_conv1_finetuned)
        print("Fine-Tuned Mean Squared Difference in Conv2: ", MSD_conv2_finetuned)


        encoder_attention_zero = self.mean_squared_difference(attentions_with_noise['model.encoder.layers.0.self_attn'], attentions_without_noise['model.encoder.layers.0.self_attn'])
        encoder_attention_last = self.mean_squared_difference(attentions_with_noise['model.encoder.layers.23.self_attn'], attentions_without_noise['model.encoder.layers.23.self_attn'])



        wandb.log({
            'entropy_conv1_with_noise': conv1_entropy_with_noise,
            'entropy_conv1_without_noise': conv1_entropy_without_noise,
            'entropy_conv2_with_noise': conv2_entropy_with_noise,
            'entropy_conv2_without_noise': conv2_entropy_without_noise,
            'MSD_conv1': MSD_conv1_finetuned,
            'MSD_conv2': MSD_conv2_finetuned,
            'MSD Attention Layer0 ': encoder_attention_zero,
            'MSD Attention Layer23 ': encoder_attention_last,
            'epoch': state.epoch
        })

noisy_data_path =  "/content/drive/MyDrive/musicml/gct634_final_project/audio_sample.wav"
non_noisy_data_path = "/content/drive/MyDrive/musicml/gct634_final_project/ariana_cafe.wav"

noisy_eval = NoisyEvaluation( noisy_data_path, non_noisy_data_path, model, processor)

Captured attention keys without noise: ['model.encoder.layers.0.self_attn_layer_norm', 'model.encoder.layers.0.self_attn.q_proj', 'model.encoder.layers.0.self_attn.k_proj', 'model.encoder.layers.0.self_attn.v_proj', 'model.encoder.layers.0.self_attn.out_proj', 'model.encoder.layers.0.self_attn', 'model.encoder.layers.1.self_attn_layer_norm', 'model.encoder.layers.1.self_attn.q_proj', 'model.encoder.layers.1.self_attn.k_proj', 'model.encoder.layers.1.self_attn.v_proj', 'model.encoder.layers.1.self_attn.out_proj', 'model.encoder.layers.1.self_attn', 'model.encoder.layers.2.self_attn_layer_norm', 'model.encoder.layers.2.self_attn.q_proj', 'model.encoder.layers.2.self_attn.k_proj', 'model.encoder.layers.2.self_attn.v_proj', 'model.encoder.layers.2.self_attn.out_proj', 'model.encoder.layers.2.self_attn', 'model.encoder.layers.3.self_attn_layer_norm', 'model.encoder.layers.3.self_attn.q_proj', 'model.encoder.layers.3.self_attn.k_proj', 'model.encoder.layers.3.self_attn.v_proj', 'model.encode

In [27]:
early_stopping = EarlyStoppingCallback(early_stopping_patience=10)
progress_callback = WandbPredictionProgressCallback(trainer, processor.tokenizer, valid_dataset, 10,1)



In [28]:
wandb.init(entity = "gct634", project = "Noisy Lyrics Transcription", name = OUTPUT_DIR
           )

wandb: Currently logged in as: xdf123 (gct634). Use `wandb login --relogin` to force relogin


In [29]:
# processor.save_pretrained(training_args.output_dir)

trainer.add_callback(progress_callback)
trainer.add_callback(early_stopping)
trainer.add_callback(noisy_eval)
print('TRAINING IN PROGRESS...')
trainer.train()
print('DONE TRAINING')
wandb.finish()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


TRAINING IN PROGRESS...


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Epoch,Training Loss,Validation Loss,Wer
1,1.900900,1.302378,59.123758
2,0.999000,1.036440,56.413731
3,0.568300,0.631665,64.588979
4,0.281200,0.641351,42.818428
5,0.225100,0.669363,52.574526
6,0.190800,0.649837,87.669377


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


...

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.ut

Baseline Convolutional Entropy (With Noise): 14.889142990112305
Baseline Convolutional Entropy (Without Noise): 14.897878646850586
Baseline Convolutional Entropy 2 (With Noise): 14.091837882995605
Baseline Convolutional Entropy 2 (Without Noise): 14.116389274597168
Baseline Mean Squared Difference in Conv1:  tensor(0.0339, device='cuda:0')
Baseline Mean Squared Difference in Conv2:  tensor(0.2013, device='cuda:0')
Fine Tuned Convolutional Entropy (With Noise): 14.8828125
Fine Tuned Convolutional Entropy (Without Noise): 14.8984375
Fine Tuned Convolutional Entropy 2 (With Noise): 14.09375
Fine Tuned Convolutional Entropy 2 (Without Noise): 14.125
Fine-Tuned Mean Squared Difference in Conv1:  tensor(0.0340, device='cuda:0', dtype=torch.float16)
Fine-Tuned Mean Squared Difference in Conv2:  tensor(0.2004, device='cuda:0', dtype=torch.float16)


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50358, 50359, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}
There were missing keys in the checkpoint model loaded: ['proj_out.weight'].
Some non-defaul

DONE TRAINING


MSD Attention Layer0,▇▇▆▆▆▆██▃▁▁▁▃▃▂▂▃▃▃
MSD Attention Layer23,▁▄▇█▆▅▆▆▅▅▄▃▃▃▄▄▄▃▄
MSD_conv1,▁▁▃▃▃▆█▆▆▆▆▆██▆████
MSD_conv2,█▅▃▁▂▃▅▅▄▃▅▄▅▅▃▃▃▂▃
entropy_conv1_with_noise,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
entropy_conv1_without_noise,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
entropy_conv2_with_noise,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
entropy_conv2_without_noise,█▁█████████████████
epoch,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
eval/loss,███▅▅▁▁▁▁▂▂▁▁▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▁▁▂▂▂▂
eval/runtime,▅▅▅▄▄▅▅▂▂▄▄██▂▂▃▃▁▁▃▃▂▂▅▅▅▂▂▂▂▄▄▃▃▁▁▂▂▃▃
